In [ ]:
!pip install opencv-python
!pip install opencv-contrib-python

In [2]:
import cv2
import torch
from ultralytics import YOLO

# Load a model
model = YOLO("yolo11n.pt")  # load a pretrained model (recommended for training)
device = torch.device("cuda" if torch.cuda.is_available() else ("mps" if torch.backends.mps.is_available() else "cpu"))

model.to(device)
# Train the model
#optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=0.0005)


In [ ]:
from pathlib import Path
import os
# Get current working directory
current_directory = Path.cwd()
# Define the relative path
relative_path = "brain_data.yaml"
# Create the full path by joining current directory and relative path
full_path = os.path.join(current_directory, relative_path)
print(full_path)
model.train(data=full_path, epochs=100, imgsz=224)
results = model.train(data=full_path, epochs=50, imgsz=224,batch=0.6,max_det=1,save=True,single_cls=True,warmup_epochs=3,val=True,plots=True,patience = 10,box=10,cls=0,konj=0)


In [56]:
trained_model=YOLO("runs/detect/train322/weights/best.pt")
import os
from train import calculate_f1
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

test_folder = 'test/'  # Replace with your folder path

# Create a list of image paths
image_paths = [os.path.join(test_folder, filename) 
               for filename in os.listdir(test_folder) 
               if filename.endswith(('.jpg', '.jpeg', '.png', '.bmp'))]  # Include desired image formats

image_paths=image_paths
test_results = trained_model(image_paths)
import json

def get_bbox_for_file_name(json_path, file_name):
    # Load the JSON file
    with open(json_path) as f:
        coco_data = json.load(f)

    # Extract relevant information
    image_id_to_file_name = {image['id']: image['file_name'] for image in coco_data['images']}
    file_name_to_image_id = {v: k for k, v in image_id_to_file_name.items()}
    annotations = coco_data['annotations']

    # Find the image ID for the specified file name
    if file_name not in file_name_to_image_id:
        raise ValueError(f"File name {file_name} not found in the dataset.")
    
    image_id = file_name_to_image_id[file_name]

    # Extract bounding boxes for the specified image ID
    bboxes = [ann['bbox'] for ann in annotations if ann['image_id'] == image_id]

    return bboxes

# Function to convert bounding boxes from (x, y, w, h) to (x_min, y_min, x_max, y_max)
def convert_bbox_format(bbox,format='min'):
    if format=='min':
        x, y, w, h = bbox
        x_min = x
        y_min = y
        x_max = x + w
        y_max = y + h
        return [x_min, y_min, x_max, y_max]
    elif format=="center":
        x_center, y_center, w, h = bbox
        x_min = x_center - w / 2
        y_min = y_center - h / 2
        x_max = x_center + w / 2
        y_max = y_center + h / 2
        return [x_min, y_min, x_max, y_max]
    else:
        raise ValueError("Invalid format. Use 'min' or 'center'.")
        
def create_bbox_mask(bbox, image_size=(640, 640)):
    mask = np.zeros(image_size, dtype=np.uint8)
    xmin, ymin, xmax, ymax = bbox
    x_min, y_min, x_max, y_max = int(xmin), int(ymin), int(xmax), int(ymax)
    mask[y_min:y_max, x_min:x_max] = 1
    return mask


# Example usage
json_path = "test/_annotations.coco.json"
# return a list of Results objects
f1_scores = []
for i,test_result in enumerate(test_results): 
    bbox_pred=test_result.boxes.xyxy.round().tolist()
    path_to_file=image_paths[i]
    file_name = os.path.basename(path_to_file)
    bbox_label = convert_bbox_format(get_bbox_for_file_name(json_path, file_name)[0],"min")
    #print(f"Predicted Bounding Box: {bbox_pred}")
    #print(f"Labeled Bounding Box: {bbox_label}")
    
    if bbox_pred:
        # Create masks for the labeled and predicted bounding boxes
        mask_pred = create_bbox_mask(bbox_pred[0])
        mask_label = create_bbox_mask(bbox_label)
        f1=calculate_f1(torch.from_numpy(mask_label), torch.from_numpy(mask_pred))
        f1_scores.append(f1)
   

    else:
        f1_scores.append(0)


    






0: 224x224 1 tumor, 16.9ms
1: 224x224 1 tumor, 16.9ms
2: 224x224 1 tumor, 16.9ms
3: 224x224 (no detections), 16.9ms
4: 224x224 2 tumors, 16.9ms
5: 224x224 1 tumor, 16.9ms
6: 224x224 2 tumors, 16.9ms
7: 224x224 1 tumor, 16.9ms
8: 224x224 1 tumor, 16.9ms
9: 224x224 1 tumor, 16.9ms
10: 224x224 1 tumor, 16.9ms
11: 224x224 1 tumor, 16.9ms
12: 224x224 1 tumor, 16.9ms
13: 224x224 1 tumor, 16.9ms
14: 224x224 1 tumor, 16.9ms
15: 224x224 1 tumor, 16.9ms
16: 224x224 1 tumor, 16.9ms
17: 224x224 1 tumor, 16.9ms
18: 224x224 1 tumor, 16.9ms
19: 224x224 (no detections), 16.9ms
20: 224x224 1 tumor, 16.9ms
21: 224x224 1 tumor, 16.9ms
22: 224x224 2 tumors, 16.9ms
23: 224x224 1 tumor, 16.9ms
24: 224x224 1 tumor, 16.9ms
25: 224x224 1 tumor, 16.9ms
26: 224x224 1 tumor, 16.9ms
27: 224x224 1 tumor, 16.9ms
28: 224x224 1 tumor, 16.9ms
29: 224x224 1 tumor, 16.9ms
30: 224x224 1 tumor, 16.9ms
31: 224x224 1 tumor, 16.9ms
32: 224x224 1 tumor, 16.9ms
33: 224x224 1 tumor, 16.9ms
34: 224x224 1 tumor, 16.9ms
35: 224x22

In [57]:
print("f1_scores",np.mean(f1_scores))
print("f1_scores",f1_scores)
#print where f1 score is less than 0.5
print("f1_scores",np.where(np.array(f1_scores)<0.5))

f1_scores 0.7657832980949955
f1_scores [0.9361939258846476, 0.84, 0.9268676690545541, 0, 0.8971259124087592, 0.0, 0.4977145759268664, 0.9489832322511594, 0.8065531083866768, 0.9279097865485301, 0.9071476888387825, 0.9646370420624153, 0.8443051201671892, 0.8539630836047775, 0.8163911082983941, 0.9447661912168648, 0.8780246913580246, 0.9303299743133768, 0.5732561434759725, 0, 0.91985559566787, 0.952174284493806, 0.7318594873185948, 0.6696914700544465, 0.9414287106519832, 0.9568480300187617, 0.8602392229268685, 0.8699731903485255, 0.9156126891975949, 0.9751364463311097, 0.6406686502078176, 0.3614341510510379, 0.7392601971601701, 0.7710426047800486, 0.9055838370983138, 0.45533740735925105, 0.9367020921061426, 0, 0.8772797527047913, 0, 0.8866515102305943, 0.555467887012782, 0.8396155295891443, 0.9590531248931002, 0.9776325364895851, 0.5635805461302311, 0.8522238163558106, 0.8521777386713595, 0.8747182672750261, 0.8224299065420562, 0.9029807130333138, 0.8335149269993463, 0.9082733812949642, 